In [ ]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('lab10.ok')

# Lab 10: Logistic Regression 

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** at the top
of your solution.

## Due Date

This assignment is due at 11:59pm Tuesday, July 30th.

In this lab you will fit a logistic regression model and evaluate using a few metrics.

# Collaborators  

Write names in this cell:

In [1]:
# Run this cell to set up your notebook
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets
import sklearn.linear_model
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline
sns.set()
sns.set_context("talk")

In this lab we will be working on the breast cancer dataset. This dataset can be easily loaded using the `sklearn.datasets.load_breast_cancer()` method. The data format is not a `pandas.DataFrame` so we will create a new DataFrame from it.

In [2]:
data = sklearn.datasets.load_breast_cancer()
# data is actually a dictionnary
print(data.keys())
print(data.DESCR)

In [3]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df.head()

Let us try to fit a simple model with only one feature.

In [4]:
# Define our features/target
X = df[["mean radius"]]
# Target data['target'] = 0 is malignant 1 is benign
Y = (data.target == 0)

In [5]:
# Split between train and test
from sklearn.model_selection import train_test_split
x_train, x_test,y_train,y_test = train_test_split(X,Y, test_size=0.25, random_state=42)

print(f"Training Data Size: {len(x_train)}")
print(f"Test Data Size: {len(x_test)}")

### Question 1

Let's first fit a logistic regression model using the training set. We will compute the training and testing accuracy, defined as:

$$
\large
\text{Training Accuracy} = \frac{1}{n_{train\_set}} \sum_{i \in \text{train_set}} {\mathbb{1}_{y_i == \hat{y_i}}}
$$

$$
\large
\text{Testing Accuracy} = \frac{1}{n_{test\_set}} \sum_{i \in \text{test_set}} {\mathbb{1}_{y_i == \hat{y_i}}}
$$

where $\hat y_i $ is the prediction of our model, $ y_i $ the true value, and $\mathbb{1}_{y_i == \hat{y_i}}$ an indicator function ($ \mathbb{1}_{y_i == \hat{y_i}} = 1 $ if and only if  $ y_i = \hat{y_i}$).

<!--
BEGIN QUESTION
name: q1
-->

In [6]:
lr = sklearn.linear_model.LogisticRegression(fit_intercept=True)

...
train_accuracy = ...
test_accuracy = ...

print(f"Train accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

In [ ]:
ok.grade("q1");

### Question 2
It seems we can a get very high test accuracy. Then how about precision and recall?  
- Precision is the fraction of instances you predicted as 1 that were actually 1.  
- Recall is the fraction of instances are actually 1 that you predicted as 1.

Precision measures the ability of the classifier not to label as positive a sample that is negative, while recall measures the ability of the classifier to find all the positive samples.

A **confusion matrix** helps us visualize/calculate the precision and recall of a classifier. Below, we compute the confusion matrix for our classifier.

We also compute the normalized confusion matrix, which is the same as the confusion matrix, but the numbers are converted to percents (sometimes it's easier to interpret percents as opposed to numbers, especially when the numbers are really big or small).

In [8]:
from sklearn.metrics import confusion_matrix

cnf_matrix = confusion_matrix(y_test, lr.predict(x_test))

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
class_names = ['False', 'True']
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

Mathematically:
$$
\text{Precision} = \frac{n_{true\_positives}}{n_{true\_positives} + n_{false\_positives}}
$$

$$
\text{Recall} = \frac{n_{true\_positives}}{n_{true\_positives} + n_{false\_negatives}}
$$

As illustrated in the figure below:
![precision_recall](precision_recall.png)

Now let's compute the precision and recall for the test set using the model we got from question 1.  
Please do not use the `sklearn.metrics` for this computation. Instead, use the formulas above.

<!--
BEGIN QUESTION
name: q2
-->

In [9]:
y_pred = lr.predict(x_test) 

precision = ...
recall = ...

print(f'precision = {precision:.4f}')
print(f'recall = {recall:.4f}')

In [ ]:
ok.grade("q2");

What can you infer from these results? Please consider the following plots, that display the distribution of the target variable in the training and testing sets.

In [12]:
fig, axes = plt.subplots(1, 2)
sns.countplot(y_train, ax=axes[0]);
sns.countplot(y_test, ax=axes[1]);

axes[0].set_title('Train')
axes[1].set_title('Test')
plt.tight_layout();

*Write your answer here, replacing this text.*

###  Question 3
Now let's try to analyze the cross entropy loss. Recall that loss would be:
$$\Large L(\theta) = -\frac{1}{n} \sum_{i=1}^n \left( y_i \log(z_i) + (1 - y_i) \log(1 - z_i)  \right) $$

where $z_i = \sigma(\theta \cdot X_i)$.

In [13]:
theta = np.array([lr.coef_[0][0],
                  lr.intercept_[0]])
X_new = np.hstack([X,
                 np.ones([len(X), 1])]) # This is adding a coefficient of 1 for the intercept term
print(theta)
print()
print(X_new)

Implement the function `lr_loss` which computes the cross-entropy loss. Note that you have been given `sigmoid`, which implements the $\sigma$ function for you.

<!--
BEGIN QUESTION
name: q3
-->

In [14]:
def sigmoid(t):
    return 1 / (1 + np.exp(-t))
def lr_loss(theta, X, Y):
    '''
    Compute the cross entropy loss using Phi, Y and theta. Hint: # The notation B @ v means: 
    compute the matrix multiplication Bv 

    Args:
        theta: The model parameters. 
        Phi: The transformed input data \phi(X)
        Y: The label 

    Return:
        The cross entropy loss.
    '''
    loss = ...
    return loss

In [ ]:
ok.grade("q3");

# You're done! Make sure to submit to okpy.

# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

In [ ]:
# Save your notebook first, then run this cell to submit.
ok.submit()